# Imports 

In [22]:
import dearpygui.dearpygui as dpg 
from AdafruitBLE_functions import connect_to_device
from AdafruitBLE_functions import write_UART_byte
from BLEMessage import BLEMessage

# BLE Message setup

In [23]:
ipg_name = 'Nordic_UART_Service'
my_connection = None
timeout = 10
message = BLEMessage(0,0,0,False)

----------
encoding b1 and b2
compression step is 1
b1 = 0, b2 = 0
----------
encoding b3
compression step is 2
b3 = 0
----------
encoding b4
compression step is 1
b4 = 0


# GUI Callback Defs

In [24]:
def button_cb(sender, app_data, user_data):
    global my_connection
    global message
    
    print('Sender is %s, app_data is %s, user_data is %s' %(sender, app_data, user_data))
    stim_amp = int(dpg.get_value(stim_amp_input))
    stim_pw = int(dpg.get_value(stim_pw_input))
    stim_fq = int(dpg.get_value(stim_fq_input))
    
    if int(dpg.get_value(stim_status_input)) == True:
        stim_status = True
    elif int(dpg.get_value(stim_status_input)) == False:
        stim_status = False
    else:
        print('Invalid input')
        
    print('Stim amplitude is %i' %(stim_amp))
    print('Stim power is %i' %(stim_pw))
    print('Stim frequency is %i' %(stim_fq))

    message.set_amp(stim_amp)
    message.set_pw(stim_pw)
    message.set_fq(stim_fq)
    message.set_status(stim_status)
    
    message_encoded = message.get_encoded()
    write_UART_byte(my_connection, message_encoded)
    
    
def connection_cb(sender, app_data, user_data):
    global my_connection
    my_connection = connect_to_device(ipg_name, timeout)
    
def disconnect_cb(sender, app_data, user_data):
    global my_connection
    my_connection.disconnect()

# GUI Setup

### Font Registry

### Value Registry

In [25]:
dpg.create_context()
dpg.create_viewport(title="Chronos", width=600, height=430)

with dpg.value_registry():
    dpg.add_bool_value(default_value=False, tag="bool_value")

In [26]:
with dpg.font_registry():
    with dpg.font("C:/Users/richa/IPG/fonts/Ubuntu-R.ttf", 100) as default_font:
        dpg.add_font_range(0x0370, 0x03FF)
    title = dpg.add_font("C:/Users/richa/IPG/fonts/Ubuntu-BI.ttf", 300)
    buttons = dpg.add_font("C:/Users/richa/IPG/fonts/Ubuntu-RI.ttf", 100)
    

### Window Setup

In [27]:
myIndent = 225
with dpg.window(tag='My Window', on_close=disconnect_cb):
    chronos = dpg.add_text('Chronos\n', color=(110, 140, 240))
    
    with dpg.group(horizontal=True, horizontal_spacing=15):  
        connect = dpg.add_button(label='connect to device', callback=connection_cb)
        disconnect = dpg.add_button(label='disconnect', callback=disconnect_cb)
        button = dpg.add_button(label='send', callback=button_cb)
        
    dpg.add_text("")
    with dpg.group(horizontal=True): 
        dpg.add_text('Stim Amplitude (\u03BCA)')
        stim_amp_input = dpg.add_input_text(default_value='0', width=65, indent=myIndent)
    with dpg.group(horizontal=True):
        dpg.add_text('Stim Pulse Width (\u03BCs)')
        stim_pw_input = dpg.add_input_text(default_value='0', width=65, indent=myIndent)
    with dpg.group(horizontal=True):
        dpg.add_text('Stim Frequency (Hz)')
        stim_fq_input = dpg.add_input_text(default_value='0', width=65, indent=myIndent)
    with dpg.group(horizontal=True):
        dpg.add_text('Start Stim')
        stim_status_input = dpg.add_checkbox(default_value=False, indent=myIndent)
    
    
    dpg.bind_font(default_font)
    dpg.bind_item_font(chronos, title)
    dpg.bind_item_font(connect, buttons)
    dpg.bind_item_font(disconnect, buttons)
    dpg.bind_item_font(button, buttons)
    dpg.set_global_font_scale(0.2)

In [28]:
# dpg.show_font_manager()
# dpg.show_style_editor()

dpg.setup_dearpygui()
dpg.show_viewport()
dpg.set_primary_window('My Window', True)
dpg.start_dearpygui()
dpg.destroy_context()